In [ ]:
using Pkg
Pkg.activate(".")
Pkg.resolve()
Pkg.instantiate()

# An Introduction to the Julia Programming Language

## CSIRO Energy Systems
## 26th May 2025


#### Credit: Slides thanks to Robin Deits

Original at: https://tinyurl.com/tech-watch-julia ([full link](https://github.com/rdeits/DetroitTechWatch2020.jl/blob/master/Intro%20to%20Julia.ipynb))

# Overview

* What is Julia?
* Tour of Julia
  * Functions, types, multiple dispatch
* Julia is fast! 
  * Benchmarking Julia vs. C and Python
* Bonus Features of Julia
  * Async tasks, multiprocessing, and metaprogramming
* What's hard to do in Julia?
* Essential Julia packages and tools

# What is Julia

<https://docs.julialang.org/en/v1/>

* Julia is a high-level language like Python with the performance of a fast language like C
* Julia is a great choice for scientific computing, with:
  * Excellent performance
  * N-dimensional arrays
  * Parallel and distributed computing
* And it's also a nice environment for general purpose programming, with:
  * An active ecosystem of packages and good tools for managing them
  * A rich type system
  * Iterators, asynchronous tasks, and coroutines


# Julia at a Glance

* First public release in 2012, version 1.0 released in 2018
* Free
    * Julia itself is [MIT licensed](https://opensource.org/license/MIT)
    * It bundles some linear algebra libraries with the GPL license which can be disabled if desired
* Built-in Just-In-Time (JIT) compiler transforms Julia code to native machine code at run time
  * Uses LLVM under the hood and works cross-platform on Windows, macOS and Linux operating systems
* Garbage collected
* Dynamically typed
* Organized via multiple dispatch

# A Tour of Julia

##  The Basics

Arithmetic:

In [ ]:
2 + (4 * 5) + sin(0.1)

Strings:

In [ ]:
# Strings
println("hello friends from all over the world")

Arrays:

In [ ]:
x = collect(1:4_000_000) # [1, 2, 3, 4]

## Functions

In [ ]:
function say_hello(name)
    return "hello $(name)"
end

In [ ]:
say_hello("Mahathir")

By default, a function is generic, so you can pass in any type you want:

In [ ]:
say_hello(1:4)

## Types

Every value in Julia has a type:

In [ ]:
typeof("hello")

In [ ]:
typeof(1) 

In [ ]:
typeof(π)

In [ ]:
typeof([1, 2, 3])

You can create your own types to organize your data:

In [ ]:
struct Person
  name::String
end

alice = Person("Alice")

Julia's types are extremely lightweight, and user-defined types are *exactly* as performant as anything built-in:

In [ ]:
sizeof(Person) == sizeof(Ptr{String})

## Multiple Dispatch

Julia does not have classes like Java, Python, or C++. Instead, code is organized around *multiple dispatch*, where the compiler chooses the appropriate method of a given function based on the types of *all* of its input arguments. 

For more, see: [The Unreasonable Effectiveness of Multiple Dispatch (Stefan Karpinski, JuliaCon 2019)](https://www.youtube.com/watch?v=kc9HwsxE1OY)

In [ ]:
greet(x, y) = println("$x greets $y")

In [ ]:
alice = Person("alice")
bob = Person("bob")

greet(alice, bob)

Currently there is only one `greet()` function, and it will work on `x` and `y` of any type:

In [ ]:
greet([1, 2, 3], "hello world")

We can use abstract types to organize the behavior of related types:

In [ ]:
abstract type Animal end

struct Cat <: Animal
    name::String
end

We've already defined `greet(x, y)` for any `x` and `y`, but we can add another definition for a more specific set of input types.

We can be as specific or as general as we like with the argument types:

In [ ]:
greet(x::Person, y::Animal) = println("$x pats 🫳 $y")

In [ ]:
greet(x::Cat, y) = println("$x meows 🐈 at $y")

Julia will always pick the *most specific* method that matches the provided function arguments.

In [ ]:
fluffy = Cat("fluffy")

greet(alice, fluffy)

In [ ]:
greet(fluffy, alice)

In [ ]:
struct Dog <: Animal
    name::String
end

greet(x::Dog, y) = println("$x 🐩 barks at $y")

greet(x::Dog, y::Person) = println("$x licks 🐶 $y's face")

greet(x::Dog, y::Dog) = println("$x sniffs $y")

In [ ]:
fido = Dog("fido")
rex = Dog("rex")

greet(alice, fido)

In [ ]:
greet(fido, fluffy)

In [ ]:
greet(fido, bob)

In [ ]:
greet(fido, rex)

If you want to know which `greet` method will be called for a given set of arguments, you can use `@which` to check:

In [ ]:
@which greet(alice, fido)

You can list all of the methods of a given function with `methods`:

In [ ]:
methods(greet)

## Modules

Modules in Julia are used to organize code into namespaces.

In [ ]:
module MyUsefulModule

export hello 

hello() = println("hello world")
goodbye() = println("goodbye world")

end

MyUsefulModule.hello()

The `using` command brings any `export`ed symbols from a module into the current namespace:

In [ ]:
using .MyUsefulModule
hello()
goodbye()


## Using Packages

Julia has a built-in package manager called `Pkg`. It handles installing packages and managing all your package environments. 

A package *environment* represents a single set of installed packages. Let's activate the environment for this talk:

In [ ]:
using Pkg
Pkg.activate(".")

(this is similar to `source venv/bin/activate` in a Python virtual environment)

We can install a package in our current environment. This will only affect that environment, so we can safely do this without breaking any other Julia projects we might be working on:

In [ ]:
Pkg.add("Colors")

The `Project.toml` file gives a concise description of the packages we've added to this environment:

In [ ]:
run(`cat Project.toml`)

The package manager also generates a complete manifest of every package that is installed, including all the transitive dependencies and their versions. You can use this to reproduce a given package environment exactly:

In [ ]:
run(`cat Manifest.toml`)

## Comparing Packages with Python
| Python    | Julia                                                                                                                                                                                        |
| --------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Pandas    | [Dataframes.jl](https://github.com/JuliaData/DataFrames.jl)                                                                                                                                  |
| Numpy     | **Base** / [Standard Library - Statistics](https://github.com/JuliaStats/Statistics.jl)                                                                                                      |
| Sklearn   | [MLJ.jl](https://github.com/alan-turing-institute/MLJ.jl) / [ScikitLearn.jl](https://github.com/cstjean/ScikitLearn.jl) / [Flux.jl](https://github.com/FluxML/Flux.jl)                       |
| Plotly    | [Plots.jl](https://github.com/JuliaPlots/Plots.jl) / [UnicodePlots.jl](https://github.com/JuliaPlots/UnicodePlots.jl)                                                                        |
| Loguru    | [Standard Library - Logging](https://docs.julialang.org/en/v1/stdlib/Logging/)                                                                                                               |
| Pickle    | [Standard Library - Serialisation](https://docs.julialang.org/en/v1/stdlib/Serialization/) / [HDF5.jl](https://github.com/JuliaIO/HDF5.jl) |
| Profiling | [Standard Library - Profiling](https://docs.julialang.org/en/v1/stdlib/Profile/)                                                                                                             |
| Requests  | [Standard Library - Downloads](https://docs.julialang.org/en/v1/stdlib/Downloads/) / [HTTP.jl](https://github.com/JuliaWeb/HTTP.jl)                                                          |
| Dask      | [Standard Library - Distributed](https://docs.julialang.org/en/v1/stdlib/Distributed/) / [Dagger.jl](https://github.com/JuliaParallel/Dagger.jl)                                             |
| unittest  | [Standard Library - Test](https://docs.julialang.org/en/v1/stdlib/Test/)                                                                                                                     |
| tqdm      | [ProgressMeter.jl](https://github.com/timholy/ProgressMeter.jl)                                                                                                                              |


## Example: Representing Colors and Images with Colors.jl

Let's take a tour of one of my favorite packages, Colors.jl, and show off some feature's of Julia's arrays along the way. 

We can load a package from the current environment with `using`:

In [ ]:
using Colors: RGB  # For now, just bring the `RGB` name into scope

In [ ]:
RGB(1, 0, 0)

The `RGB` type from Colors.jl knows how to render itself as an actual colored `div` when running in Jupyter. We can also print its value as a string if we want:

In [ ]:
print(RGB(1, 0, 0))

Julia arrays are fully generic, so we can create an array of colors:

In [ ]:
C = [RGB(i, j, 0) for i in 0:0.1:1, j in 0:0.1:1]

In [ ]:
typeof(C)

C is an array like any other, so we can index into it and slice it:

In [ ]:
 C[8, 2]

In [ ]:
C[:, 6]

Let's pull out the red channel from our image `C`:

In [ ]:
using Colors: red, green, blue

If you don't know what a function does, you can use the `?` operator to access its docstring:

In [ ]:
?red

In [ ]:
red(C[8, 8])

### Broadcasting

To get the red channel of each element of `C`, we can use *broadcasting*. The syntax `f.(x)` applies the function `f` to each element of `x`:

In [ ]:
red.(C)

That's not very visual. Let's render that red channel as a grayscale image:

In [ ]:
using Colors: Gray

Gray.(red.(C))

Or we can put get red channel back:

In [ ]:
RGB.(red.(C),0,0)

Julia's broadcasting provides guaranteed *loop fusion*. That means that if you do `Gray.(red.(x))`, the language guarantees that it will do only one loop over the elements of `x`, computing `Gray(red(x_i))` for each `x_i` in `x`. 

See https://julialang.org/blog/2017/01/moredots/ for more. 

# Julia is Fast

* I claimed at the beginning of this talk that Julia has performance on par with C. Let's prove it!
* To show this, I'll implement the basic `sum` function in Julia, C, and Python so we can compare them:

Let's start with Julia:

In [ ]:
"""
Naive implementation of sum. Works for any iterable `x` with any element type.
"""
function my_sum(x)
    result = zero(eltype(x))
    for element in x
        result += element
    end
    return result
end

And let's create some data to test with:

In [ ]:
data = rand(Float64, 10^7)

To measure the performance of `my_sum`, we'll use the BenchmarkTools.jl package. 

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark my_sum($data)

In this case, we only care about the minimum time. The `@btime` macro is a shorthand to print just that minimum time:

In [ ]:
@btime my_sum($data)

Let's compare this with C. It's easy to call functions from C shared libraries in Julia:

In [ ]:
"""
Call the `strcmp` function from `libc.so`
"""
function c_compare(x::String, y::String)
    # We have to tell the compiler that this C function returns an `int` and 
    # expects two `char *` inputs. The `Cint` and `Cstring` types are convenient
    # shorthands for those:
    ccall(:strcmp, Cint, (Cstring, Cstring), x, y)
end

In [ ]:
c_compare("hello", "hello")

Calling C functions has very little overhead:

In [ ]:
@btime c_compare($("hello"), $("hello"))

Let's create a C implementation of `my_sum`. We can do that without leaving Julia by piping some code directly to GCC:

In [ ]:
C_code = """

#include <stddef.h>  // For `size_t`

// Note: our Julia code works for any type, but the C implementation 
// is only for `double`.

double c_sum(size_t n, double *X) {
    double s = 0.0;
    size_t i;
    for (i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}

""";

Now let's generate a name for our shared library:

In [ ]:
# dlext gives the correct file extension for a shared library on this platform
using Libdl: dlext
const Clib = tempname() * "." * dlext

To send the code to GCC, we can use `open()` on a command to write directly to the `stdin` of that command as if it were any other file- or buffer-like object:

In [ ]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do cmd
    print(cmd, C_code) 
end

Now we can define a Julia function that calls the C function we just compiled:

In [ ]:
# The return type and argument types must match the signature we declared above:
# 
#   double c_sum(size_t n, double *X) 
# 
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Cdouble, (Csize_t, Ptr{Cdouble}), length(X), X)

Now let's measure the performance of the pure C function:

In [ ]:
@btime c_sum($data)

Let's plot the result using the Plots.jl package:

In [ ]:
using Plots

In [ ]:
results = [
    "my_sum (Julia)" => 8.569,
    "c_sum (C)" => 8.52
]

bar(first.(results), last.(results), xlabel="function", ylabel="time (ms, shorter is better)", legend=nothing)

Our naive Julia code is just as fast as our naive C code! 

Is that as fast as we can go? What about Julia's built-in `sum()` function:

In [ ]:
@btime sum($data)

In [ ]:
results = [
    "my_sum (Julia)" => 8.569,
    "c_sum (C)" => 8.52,
    "sum (Julia)" => 1.3,
]

bar(first.(results), last.(results), xlabel="function", ylabel="time (ms, shorter is better)", legend=nothing)

What's going on? Is the `sum()` function using some built-in behavior we don't have access to?

Nope--we can achieve that result easily with a few modifications:

In [ ]:
function my_fast_sum(x)
    result = zero(eltype(x))
    
    # `@inbounds` is a macro which disables all bounds checking within a given block. 
    #
    # `@simd` enables additional vector operations by indicating that it is OK to potentially
    # evaluate the loop out-of-order. 
    @inbounds @simd for element in x
        result += element
    end
    result
end

In [ ]:
@btime my_fast_sum($data)

In [ ]:
results = [
    "my_sum (Julia)" => 8.569,
    "c_sum (C)" => 8.52,
    "sum (Julia)" => 1.3,
    "my_fast_sum (Julia)" => 1.28,
]

bar(first.(results), last.(results), xlabel="function", ylabel="time (ms, shorter is better)", legend=nothing)

With some pretty simple changes, we were able to create a pure-Julia function which is twice as fast as our naive C function while still being clear and completely generic:

In [ ]:
my_fast_sum([1, 2.5, π])

Just for reference, let's compare with Python. It's easy to call Python code from Julia too--we just need the `PyCall` package:

In [ ]:
using PyCall

In [ ]:
py_math = pyimport("math")
py_math.sin(1.0)

Just as we did with C, we can quickly define a Python sum function without leaving Julia:

In [ ]:
# The PyCall package lets us define python functions directly from Julia:

py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

# mysum_py is a reference to the Python mysum function
py_sum = py"""mysum"""o

Let's make sure we're getting similar answers everywhere:

In [ ]:
py_sum(data) ≈ c_sum(data) ≈ sum(data) ≈ my_sum(data) ≈ my_fast_sum(data)

In [ ]:
@btime py_sum($data)

In [ ]:
results = [
    "my_sum (Julia)" => 8.569,
    "c_sum (C)" => 8.52,
    "sum (Julia)" => 1.3,
    "my_fast_sum (Julia)" => 1.28,
    "py_sum (Python)" => 443.4,
]

bar(first.(results), last.(results), xlabel="function", ylabel="time (ms, shorter is better)", legend=nothing)

### What about Numpy or Cython?

* Of course, there are faster ways to sum a vector of `double`s in Python than a `for` loop. 
* `numpy.sum()` is just as fast as Julia's `sum()` for large vectors...
* ...but there are some caveats:
  * NumPy is only efficient for a pre-determined set of numeric types. 
  * NumPy cannot be extended without switching into an entirely different programming language, build system, and code environment. 
  * So, if `numpy.sum()` happens to cover the cases you actually need, then go for it!
  * But if you want to be able to write efficient code that does not happen to cover the specific set of functions and types in NumPy, then you need Julia. 

In [ ]:
struct Point{T}
    x::T
    y::T
end

function Base.zero(::Type{Point{T}}) where {T} 
    Point{T}(zero(T), zero(T))
end
    
Base.:+(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)

points = [Point(rand(), rand()) for _ in 1:10^7];

In [ ]:
@btime my_fast_sum($points)

@code_native my_fast_sum(points)

# Bonus Features of Julia

## Asynchronous Tasks

Julia supports asynchronous cooperative tasks, with `libuv` providing the backend. These tasks are great for handling operations like IO or network requests:

In [ ]:
using HTTP: request

In [ ]:
for i in 1:5
    @async begin
        println("starting request $i")
        r = request("GET", "https://jsonplaceholder.typicode.com/posts/$i")
        println("got response $i with status $(r.status)")
    end
end
        

## Multi-Threading


Julia also supports parallel and distributed computing (see https://docs.julialang.org/en/v1/manual/parallel-computing/ for more). In addition, Julia 1.3 implemented a new feature, the Parallel Task Run-Time (PATR), which allows for *composable* multi-threading. It is now possible for a parallelized Julia function to call other parallelized code without over-subsubscribing the available processors. See 
https://julialang.org/blog/2019/07/multithreading/ for more. 

In [ ]:
using Base.Threads: @spawn


function fib(n::Int)
    if n < 2
        return n
    end
    # `@spawn` creates a new parallel task. Tasks are lightweight and can be
    # created at will. The Julia Parallel Task Run-Time handles scheduling the
    # tasks to native threads in a depth first manner. That means that you can
    # write parallel code which calls other parallel code without over-subscribing
    # your available processors. 
    t = @spawn fib(n - 2)
    return fib(n - 1) + fetch(t)
end 

In [ ]:
println("Number of threads = $(Threads.nthreads())")
a = zeros(10)
Threads.@threads for i = 1:10
    a[i] = Threads.threadid()
end
println("Thread IDs: $a")

## No Implicit Copying

Values are never copied unless you intentionally copy or convert them. That means that functions can mutate their input arguments to efficiently do work in-place:

In [ ]:
"""
Invert the sign of the vector `x`, operating in-place to avoid any memory allocation.
"""
function invert!(x::AbstractVector)
    for i in eachindex(x)
        x[i] = -x[i]
    end
end

Note: the `!` in the function name is just a convention: it signals to readers of the code
that the input argument `x` will be modified.

In [ ]:
x = [1, 2, 3]
invert!(x)
x

In [ ]:
@btime invert!($x)

## Anything Can Be a Value

Julia has no special rules about what can or cannot be assigned to a variable or passed to a function. 

### Functions are Values

A Julia function is a value like any other, so passing functions around and implementing higher-order functions is trivial:

In [ ]:
"""
map_reduce: apply `operator` to each element in `array` and reduce pairwise via `reduction`
"""
function map_reduce(operator, reduction, array, initial_value)
    result = initial_value
    for item in array
        result = reduction(result, operator(item))
    end
    result
end

In [ ]:
map_reduce(sin, +, [1, 2, 3, 4], 0)

We can define `sum` in terms of `map_reduce`:

In [ ]:
fancy_sum(x) = map_reduce(identity, +, x, zero(eltype(x)))

The performance is just as good as our hand-written `sum` loop:

In [ ]:
@btime fancy_sum($data)

To get all the way down to 5ms, we'd need to apply the same `@inbounds` and `@simd` annotations.  

### Types are Values

Types can also be passed around as values and bound to variables with no special rules. This makes implementing factories or constructors easy:

In [ ]:
function empty_matrix(T::Type, rows::Integer, cols::Integer)
    zeros(T, rows, cols)
end

In [ ]:
empty_matrix(Int, 3, 3)

In [ ]:
empty_matrix(Point{Float64}, 3, 3)

### Expressions are Values

Even the expressions that representing Julia code are represented as values in Julia. You can create an expression with the `:()` operator, and you can inspect it just like any other object. 

In [ ]:
expr = :(1 + 2)

An expression has a `head` indicating what type of expression it is and zero or more `args`:

In [ ]:
expr.head

In [ ]:
expr.args

## Metaprogramming

Since expressions are just values, we can easily write functions to manipulate them:

In [ ]:
switch_to_subtraction!(x::Any) = nothing

"""
Change all `+` function calls to `-` function calls. 

<sarcasm>
Great for fixing sign errors in your code!
</sarcasm>
"""
function switch_to_subtraction!(ex::Expr)
    if ex.head == :call && ex.args[1] == :(+)
        ex.args[1] = :(-)
    end
    for i in 2:length(ex.args)
        switch_to_subtraction!(ex.args[i])
    end
end

In [ ]:
expr = :((1 + 2) * (3 + 4) * sqrt(2))

In [ ]:
switch_to_subtraction!(expr)

expr

### Macros

A macro is written just like a normal Julia function. The difference is that a macro operates on the *expression* itself, not on its value:

In [ ]:
"""
Modify a given expression, replacing all string literals with "cat"
"""
macro more_cats(expr)
    for i in eachindex(expr.args)
        if expr.args[i] isa String
            expr.args[i] = "cat"
        end
    end
    return esc(expr)
end

Macros are always called with the `@` prefix in Julia:

In [ ]:
@more_cats println("hello world")

`@macroexpand` shows the code that another macro will generate:

In [ ]:
@macroexpand @more_cats println("hello world")

### Actually Useful Julia Macros

`@show` : print out the *name* of a variable and its value. Great for quick debugging:

In [ ]:
x = 5
@show x

`@time` measure the elapsed time of an expression and return the result of that expression:

In [ ]:
@time sqrt(big(π))

`@showprogress`: Time each iteration of a loop and estimate how much longer it will take to finish:

In [ ]:
using ProgressMeter: @showprogress

In [ ]:
@showprogress for i in 1:100
    sum(rand(10^7))
end

# What's Hard to Do in Julia?

What is the compiler team working on making better? https://discourse.julialang.org/t/compiler-work-priorities/17623

What are some subtle problems that the Julia team working on improving?




## Compiler Latency

* The JIT compiler runs each time it sees a function being called with a new input type. 
* That makes the first call to every function slow, since you have to wait for the JIT.
  * This makes Julia awkward to use for things like shell scripts or AWS lambda
* Pre-compilation has very much improved this issue in recent years.

## Static Compilation

* To avoid the JIT lag, you can compile a Julia package to a standalone executable using [PackageCompiler.jl](https://github.com/JuliaLang/PackageCompiler.jl), but:
    * This workflow is still under development, and you may sometimes run into interesting bugs
    * The resulting libraries tend to be quite large

## Embedded Computing

* It can be hard to run Julia on memory-limited systems, since you need the compiler living alongside your code. 
* Static compilation can help, but this isn't a well-developed workflow yet.

## Static Analysis

* There are some linting tools for Julia (like the `vscode-julia` extension for Visual Studio Code), but they are not as mature as languages like Python, C, Java, etc.
* Static analysis of Julia is harder, since the language itself is dynamically typed. 
  * To be fair, static analysis of C++ is [undecidable](https://blog.reverberate.org/2013/08/parsing-c-is-literally-undecidable.html) but we still have tools that do a pretty good job most of the time. 

# Useful Julia Tools

## Julia-VSCode

https://github.com/julia-vscode/julia-vscode

* Code highlighting, snippets, linting, and completions
* Integrated plot and table viewers
* General extension support via the VSCode language server

<img src="img/vscode.png" width="700px"></img>

## Flux.jl

https://fluxml.ai/Flux.jl/stable/

* Flexible library for machine learning built entirely in Julia
* Feed-forward and recurrent neural nets
* Gradients via automatic differentiation
* GPU support via CuArrays.jl

```julia
m = Chain(
  Dense(784, 32, σ),
  Dense(32, 10), softmax
)

loss(x, y) = Flux.mse(m(x), y)
ps = Flux.params(m)

for i in 1:num_training_iters
    Flux.train!(loss, ps, data, opt)
end
```

## DifferentialEquations.jl

https://github.com/SciML/DifferentialEquations.jl

* {stochastic | deterministic | ordinary | partial} differential equations
* Automatic differentiation and sparsity detection
* GPU support
* Sensitivity analysis and parameter estimation
* Access to pure-Julia solvers and existing C and Fortran solvers


<img src="img/DifferentialEquations_Example.png" width="500px"></img>

## DataFrames.jl

https://github.com/JuliaData/DataFrames.jl

* In-memory tabular data
* Joining, indexing, grouping, and split-apply-combine

```julia
julia> using DataFrames

julia> df = DataFrame(A = 1:4, B = ["M", "F", "F", "M"])
4×2 DataFrame
│ Row │ A     │ B      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 1     │ M      │
│ 2   │ 2     │ F      │
│ 3   │ 3     │ F      │
│ 4   │ 4     │ M      │
```

## JuMP.jl

https://github.com/JuliaOpt/JuMP.jl

* Continuous and discrete optimization
* Support for a wide variety of free and commercial solvers
* Efficient high-level language for mathematical programming


Example: Solving a simple model-predictive control problem as a quadratic program ([source](https://github.com/rdeits/DynamicWalking2018.jl/blob/master/notebooks/6.%20Optimization%20with%20JuMP.ipynb)):

[![demo of a simple model-predictive control problem](img/mpc.gif)](https://github.com/rdeits/DynamicWalking2018.jl/blob/master/notebooks/6.%20Optimization%20with%20JuMP.ipynb)



# Where To Go Next?

* Download Julia from https://julialang.org/
* Check out the manual at https://docs.julialang.org/en/v1/
* Ask questions on [discourse](https://discourse.julialang.org/) and [slack](https://slackinvite.julialang.org/)
* Find interesting packages on [juliahub](https://juliahub.com/ui/Home)

![triangulated background](img/julia-triangle-background.svg)

[Logo by Cormullion and David P. Saunders](https://nbviewer.jupyter.org/github/dpsanders/JuliaCon2019_tshirt/blob/master/penroseiana.ipynb)